In [3]:
#Create a list of viruses with unique family, genus, tnumber, genome combinations - resolution breaks ties
#Uses res.txt which stores resolution info and writes to uniq_lres.xlsx
import json
with open('db.json', 'r') as f:
    data = json.load(f)


fams = data['filters']['family']

gens = data['filters']['genus']

tnums = data['filters']['tnumber']

genomes = data['filters']['genome']

reslist = []

filters = {'family': fams, 'genus': gens,'tnumber': tnums,'genome': genomes}

with open('res.txt', 'r') as f:
    id_res = {l.split()[0]:l.split()[1] for l in f}

el_lists = []
for k in data['data']:
    el = data['data'][k]
    if 'family' in el.keys() and 'genome' in el.keys() and 'genus' in el.keys() and 'tnumber' in el.keys():
        el_lists.append([el['family'], el['genus'], el['tnumber'], el['genome'], k])
        
el_lists_red = [e[:4] for e in el_lists]

def find_best(tup_list, el):
    first = True
    blist = []
    for t in tup_list:
        res = t[0]
        inf = t[1]
        if inf[:-1] == el[:-1]:
            if first or res < best[-1]: 
                best = inf + [res]
                first = False
    return best

res_tups = []
ids = []
reslist = []
for e in el_lists:
    if e[:4] not in reslist:
        reslist.append(e[:4])
        k = e[4]
        res = float(id_res[k] if k in id_res and id_res[k] is not None else 999)
        res_tups.append((res, e))
        ids.append(k)

for i in range(len(reslist)):
    reslist[i].extend([ids[i]])

lres = []
for el in reslist:
    b = find_best(res_tups, el)
    if b not in lres: lres.append(b)

columns = ['family', 'genus', 'tnumber', 'genome', 'id', 'resolution']

dfdict = {}

for i in range((len(columns))):
    es = [e[i] for e in lres]
    dfdict[columns[i]] = es

from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd

xl_file = 'uniq_lres.xlsx'
df = pd.DataFrame(dfdict)
df.to_excel(xl_file, index = False)